<a href="https://colab.research.google.com/github/JotaBlanco/Pernas/blob/main/Bc3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
def exportar_partidas(df):
  df = df[df["Nat"]=="Partida"]
  df["Descripción"] = df["Descripción"].str.replace('\n', ' ', regex=False)
  df.to_csv("df_partida.csv", index=False)

In [3]:
def import_df(location):
    df = pd.read_csv(location)
    df.columns = ["Código", "Nat", "Ud", "Resumen", "CanPres", "Pres", "ImpPres"]
    df = df.iloc[2:]
    return df

def mover_descripciones(df):
    df["Descripción"] = None
    mask = df[["Código", "Nat", "Ud"]].isna().all(axis=1)
    for idx in df[mask].index:
        if idx > 0:
            df.at[idx - 1, "Descripción"] = df.at[idx, "Resumen"]
    return df[~mask].reset_index(drop=True)

def expandir_codigo(df):
    # Divide el contenido de 'Código' por "." en hasta 3 partes
    nuevos_codigos = df["Código"].astype(str).str.split(".", n=2, expand=True)
    nuevos_codigos.columns = ["CO1", "CO2", "CO3"]

    # Convierte a numérico (opcional, convierte vacíos a NaN)
    nuevos_codigos = nuevos_codigos.apply(pd.to_numeric, errors='coerce')

    # Encuentra la posición de la columna 'Código'
    idx = df.columns.get_loc("Código") + 1

    # Inserta las columnas nuevas justo después de 'Código'
    for i, col in enumerate(["CO1", "CO2", "CO3"]):
        df.insert(idx + i, col, nuevos_codigos[col])

    return df

def mantener_solo_partidas(df):
    df = df[df["Nat"]=="Partida"]
    return df

## 01 - Importar bc3s
Importar archivos a comparar

In [4]:
url_actualizado = "https://raw.githubusercontent.com/JotaBlanco/Pernas/refs/heads/main/Actualizado.csv"
url_original = "https://raw.githubusercontent.com/JotaBlanco/Pernas/refs/heads/main/Original.csv"

In [5]:
df0 = import_df(url_original)
display(df0.head())
df0 = mover_descripciones(df0)
#df0 = expandir_codigo(df0)
df0 = mantener_solo_partidas(df0)
display(df0.head())

,Código,Nat,Ud,Resumen,CanPres,Pres,ImpPres
2,01,Capítulo,NaN,Acondicionamiento del terreno,1,0.00,0.00
3,NaN,NaN,NaN,El presente presupuesto prevalecerán ante cont...,NaN,NaN,NaN
4,01.01,Capítulo,NaN,Movimiento de tierras en edificación,1.000,0.00,0.00
5,01.01.01,Partida,m³,Excavación de terreno flojo,40.800,0.00,0.00
6,NaN,NaN,NaN,Excavación de hasta profundidad necesaria segú...,NaN,NaN,NaN


,Código,Nat,Ud,Resumen,CanPres,Pres,ImpPres,Descripción
2,01.01.01,Partida,m³,Excavación de terreno flojo,40.800,0.00,0.00,Excavación de hasta profundidad necesaria segú...
3,01.01.02,Partida,m³,Excavación de sótanos roca dura,43.605,0.00,0.00,Excavación de sótanos de hasta profundidad nec...
4,01.01.03,Partida,m³,Excavación de zanjas para instalaciones,24.750,0.00,0.00,Excavación de zanjas para instalaciones hasta ...
5,01.01.04,Partida,m³,Relleno envolvente y principal de zanjas para ...,17.303,0.00,0.00,Relleno envolvente y principal de zanjas para ...
6,01.01.05,Partida,m³,Relleno con suelo seleccionado externo,190.540,0.00,0.00,Relleno compactado con suelo seleccionado pro...


In [6]:
df1 = import_df(url_actualizado)
display(df1.head())
df1 = mover_descripciones(df1)
#df1 = expandir_codigo(df1)
df1 = mantener_solo_partidas(df1)
display(df1.head())

,Código,Nat,Ud,Resumen,CanPres,Pres,ImpPres
2,01,Capítulo,NaN,Acondicionamiento del terreno,1,0.00,0.00
3,NaN,NaN,NaN,El presente presupuesto prevalecerán ante cont...,NaN,NaN,NaN
4,01.01,Capítulo,NaN,Movimiento de tierras en edificación,1.000,0.00,0.00
5,01.01.01,Partida,m³,Excavación de terreno flojo,40.800,0.00,0.00
6,NaN,NaN,NaN,Excavación de hasta profundidad necesaria segú...,NaN,NaN,NaN


,Código,Nat,Ud,Resumen,CanPres,Pres,ImpPres,Descripción
2,01.01.01,Partida,m³,Excavación de terreno flojo,40.800,0.00,0.00,Excavación de hasta profundidad necesaria segú...
3,01.01.02,Partida,m³,Excavación de sótanos roca dura,43.605,0.00,0.00,Excavación de sótanos de hasta profundidad nec...
4,01.01.03,Partida,m³,Excavación de zanjas para instalaciones,24.750,0.00,0.00,Excavación de zanjas para instalaciones hasta ...
5,01.01.04,Partida,m³,Relleno envolvente y principal de zanjas para ...,17.303,0.00,0.00,Relleno envolvente y principal de zanjas para ...
6,01.01.05,Partida,m³,Relleno con suelo seleccionado externo,471.650,0.00,0.00,Relleno compactado con suelo seleccionado pro...


## 02 - Cruzar y encontrar cambios

In [13]:
df0.head()

,Código,Nat,Ud,Resumen,CanPres,Pres,ImpPres,Descripción
2,01.01.01,Partida,m³,Excavación de terreno flojo,40.800,0.00,0.00,Excavación de hasta profundidad necesaria segú...
3,01.01.02,Partida,m³,Excavación de sótanos roca dura,43.605,0.00,0.00,Excavación de sótanos de hasta profundidad nec...
4,01.01.03,Partida,m³,Excavación de zanjas para instalaciones,24.750,0.00,0.00,Excavación de zanjas para instalaciones hasta ...
5,01.01.04,Partida,m³,Relleno envolvente y principal de zanjas para ...,17.303,0.00,0.00,Relleno envolvente y principal de zanjas para ...
6,01.01.05,Partida,m³,Relleno con suelo seleccionado externo,190.540,0.00,0.00,Relleno compactado con suelo seleccionado pro...


In [14]:
df0.columns

Index(['Código', 'Nat', 'Ud', 'Resumen', 'CanPres', 'Pres', 'ImpPres',
       'Descripción'],
      dtype='object')

In [16]:
df_both = pd.merge(df0, df1, on=["Código", "Nat"], how="outer", suffixes=("", "_2"))
print(len(df0), len(df1), len(df_both))
df_both.head()

365 367 371


,Código,Nat,Ud,Resumen,CanPres,Pres,ImpPres,Descripción,Ud_2,Resumen_2,CanPres_2,Pres_2,ImpPres_2,Descripción_2
0,01.01.01,Partida,m³,Excavación de terreno flojo,40.800,0.00,0.00,Excavación de hasta profundidad necesaria segú...,m³,Excavación de terreno flojo,40.800,0.00,0.00,Excavación de hasta profundidad necesaria segú...
1,01.01.02,Partida,m³,Excavación de sótanos roca dura,43.605,0.00,0.00,Excavación de sótanos de hasta profundidad nec...,m³,Excavación de sótanos roca dura,43.605,0.00,0.00,Excavación de sótanos de hasta profundidad nec...
2,01.01.03,Partida,m³,Excavación de zanjas para instalaciones,24.750,0.00,0.00,Excavación de zanjas para instalaciones hasta ...,m³,Excavación de zanjas para instalaciones,24.750,0.00,0.00,Excavación de zanjas para instalaciones hasta ...
3,01.01.04,Partida,m³,Relleno envolvente y principal de zanjas para ...,17.303,0.00,0.00,Relleno envolvente y principal de zanjas para ...,m³,Relleno envolvente y principal de zanjas para ...,17.303,0.00,0.00,Relleno envolvente y principal de zanjas para ...
4,01.01.05,Partida,m³,Relleno con suelo seleccionado externo,190.540,0.00,0.00,Relleno compactado con suelo seleccionado pro...,m³,Relleno con suelo seleccionado externo,471.650,0.00,0.00,Relleno compactado con suelo seleccionado pro...


In [18]:
df_cambios = pd.DataFrame()

for col in ['Ud', 'Resumen', 'CanPres', 'Pres', 'ImpPres', 'Descripción']:
  filter_i = df_both[col] != df_both[col + "_2"]
  df_cambios[col] = filter_i

df_cambios["CAMBIO"] = df_cambios.any(axis=1)
df_cambios

,Ud,Resumen,CanPres,Pres,ImpPres,Descripción,CAMBIO
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...
366,False,False,False,False,False,False,False
367,False,False,False,False,False,False,False
368,False,False,False,False,False,False,False
369,False,False,False,False,False,False,False


In [21]:
df_cambios.sum()

,0
Ud,16
Resumen,59
CanPres,40
Pres,10
ImpPres,10
Descripción,41
CAMBIO,94


In [20]:
df_cambios.sum()/len(df_cambios)

,0
Ud,0.043127
Resumen,0.159030
CanPres,0.107817
Pres,0.026954
ImpPres,0.026954
Descripción,0.110512
CAMBIO,0.253369


# 03 - Importar presupuestos

In [22]:
!pip install --upgrade gspread gspread_dataframe

In [30]:
import gspread
from google.colab import auth
auth.authenticate_user()

from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [31]:
spreadsheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1ykKzP3h3NsNxd0iO0-5weY8CF523izX17iYF-NHZSXQ/edit?gid=794912994#gid=794912994")
spreadsheet

<Spreadsheet 'Pernas 5' id:1ykKzP3h3NsNxd0iO0-5weY8CF523izX17iYF-NHZSXQ>

In [32]:
# Get a specific worksheet (by name)
worksheet = spreadsheet.worksheet("Presupuestos ORIG")
worksheet

<Worksheet 'Presupuestos ORIG' id:794912994>

In [33]:
from gspread_dataframe import get_as_dataframe
row_where_it_starts = 5

df = get_as_dataframe(worksheet, evaluate_formulas=True)
df.columns = df.iloc[row_where_it_starts-2, :].values
df = df.iloc[row_where_it_starts-1:].dropna(how='all').reset_index(drop=True)
df

,Código,Nat,Ud,Resumen,CanPres,Pres,ImpPres,Descripción,Cap,Castro € unitario,Castro € tot,Cap,Balboa € unitario,Balboa € tot,NaN
0,01.01.01,Partida,m³,Excavación de terreno flojo,40.8,0,0,Excavación de hasta profundidad necesaria segú...,1.001,28.41,1159.128,01.01.01,36.53,1490.424,NaN
1,01.01.02,Partida,m³,Excavación de sótanos roca dura,43.605,0,0,Excavación de sótanos de hasta profundidad nec...,1.002,85.22,3716.0181,01.01.02,51.82,2259.6111,NaN
2,01.01.03,Partida,m³,Excavación de zanjas para instalaciones,24.75,0,0,Excavación de zanjas para instalaciones hasta ...,1.003,41.99,1039.2525,01.01.03,28.41,703.1475,NaN
3,01.01.04,Partida,m³,Relleno envolvente y principal de zanjas para ...,17.303,0,0,Relleno envolvente y principal de zanjas para ...,1.004,18.53,320.62459,01.01.04,14.96,258.85288,NaN
4,01.01.05,Partida,m³,Relleno con suelo seleccionado externo,190.54,0,0,Relleno compactado con suelo seleccionado pro...,1.005,65.15,12413.681,01.01.05,27.15,5173.161,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,YSS034,Partida,Ud,"Señal de evacuación, salvamento y socorro, de ...",1,0,0,"Señal de evacuación, salvamento y socorro, de ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
361,YSM005,Partida,m,"Cinta de señalización, de material plástico, d...",10,0,0,"Cinta de señalización, de material plástico, d...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
362,YSM006,Partida,m,"Doble cinta de señalización, de material plást...",10,0,0,"Doble cinta de señalización, de material plást...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
363,YSM010,Partida,m,Malla de señalización de polietileno de alta d...,10,0,0,Malla de señalización de polietileno de alta d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
df.columns

Index([           'Código',               'Nat',                'Ud',
                 'Resumen',           'CanPres',              'Pres',
                 'ImpPres',       'Descripción',               'Cap',
       'Castro € unitario',      'Castro € tot',               'Cap',
       'Balboa € unitario',      'Balboa € tot',                 nan],
      dtype='object')

# 04 - Gasto Cambios

In [41]:
def safe_diff_castro(row):
    try:
        delta = float(row["CanPres_2"]) - float(row["CanPres"])
        return delta * float(row["Castro € unitario"])
    except:
        return np.nan


def safe_diff_balboa(row):
    try:
        delta = float(row["CanPres_2"]) - float(row["CanPres"])
        return delta * float(row["Balboa € unitario"])
    except:
        return np.nan


In [42]:
cols = ['Código', 'Castro € unitario', 'Castro € tot', 'Balboa € unitario', 'Balboa € tot']
df_precios = pd.merge(df_both, df[cols], on="Código", how="left")
df_precios["Delta Castro"] = df_precios.apply(safe_diff_castro, axis=1)
df_precios["Delta Balboa"] = df_precios.apply(safe_diff_balboa, axis=1)
df_precios

,Código,Nat,Ud,Resumen,CanPres,Pres,ImpPres,Descripción,Ud_2,Resumen_2,CanPres_2,Pres_2,ImpPres_2,Descripción_2,Castro € unitario,Castro € tot,Balboa € unitario,Balboa € tot,Delta Castro,Delta Balboa
0,01.01.01,Partida,m³,Excavación de terreno flojo,40.800,0.00,0.00,Excavación de hasta profundidad necesaria segú...,m³,Excavación de terreno flojo,40.800,0.00,0.00,Excavación de hasta profundidad necesaria segú...,28.41,1159.128,36.53,1490.424,0.0000,0.0000
1,01.01.02,Partida,m³,Excavación de sótanos roca dura,43.605,0.00,0.00,Excavación de sótanos de hasta profundidad nec...,m³,Excavación de sótanos roca dura,43.605,0.00,0.00,Excavación de sótanos de hasta profundidad nec...,85.22,3716.0181,51.82,2259.6111,0.0000,0.0000
2,01.01.03,Partida,m³,Excavación de zanjas para instalaciones,24.750,0.00,0.00,Excavación de zanjas para instalaciones hasta ...,m³,Excavación de zanjas para instalaciones,24.750,0.00,0.00,Excavación de zanjas para instalaciones hasta ...,41.99,1039.2525,28.41,703.1475,0.0000,0.0000
3,01.01.04,Partida,m³,Relleno envolvente y principal de zanjas para ...,17.303,0.00,0.00,Relleno envolvente y principal de zanjas para ...,m³,Relleno envolvente y principal de zanjas para ...,17.303,0.00,0.00,Relleno envolvente y principal de zanjas para ...,18.53,320.62459,14.96,258.85288,0.0000,0.0000
4,01.01.05,Partida,m³,Relleno con suelo seleccionado externo,190.540,0.00,0.00,Relleno compactado con suelo seleccionado pro...,m³,Relleno con suelo seleccionado externo,471.650,0.00,0.00,Relleno compactado con suelo seleccionado pro...,65.15,12413.681,27.15,5173.161,18314.3165,7632.1365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,YSS031,Partida,Ud,"Señal de prohibición, de PVC serigrafiado, de ...",1.000,0.00,0.00,"Señal de prohibición, de PVC serigrafiado, de ...",Ud,"Señal de prohibición, de PVC serigrafiado, de ...",1.000,0.00,0.00,"Señal de prohibición, de PVC serigrafiado, de ...",NaN,NaN,NaN,NaN,NaN,NaN
367,YSS032,Partida,Ud,"Señal de obligación, de PVC serigrafiado, de 2...",1.000,0.00,0.00,"Señal de obligación, de PVC serigrafiado, de 2...",Ud,"Señal de obligación, de PVC serigrafiado, de 2...",1.000,0.00,0.00,"Señal de obligación, de PVC serigrafiado, de 2...",NaN,NaN,NaN,NaN,NaN,NaN
368,YSS033,Partida,Ud,"Señal de extinción, de PVC serigrafiado, de 29...",1.000,0.00,0.00,"Señal de extinción, de PVC serigrafiado, de 29...",Ud,"Señal de extinción, de PVC serigrafiado, de 29...",1.000,0.00,0.00,"Señal de extinción, de PVC serigrafiado, de 29...",NaN,NaN,NaN,NaN,NaN,NaN
369,YSS034,Partida,Ud,"Señal de evacuación, salvamento y socorro, de ...",1.000,0.00,0.00,"Señal de evacuación, salvamento y socorro, de ...",Ud,"Señal de evacuación, salvamento y socorro, de ...",1.000,0.00,0.00,"Señal de evacuación, salvamento y socorro, de ...",NaN,NaN,NaN,NaN,NaN,NaN
